<a href="https://colab.research.google.com/github/fakhrulfaiz/hands_on_machine_learning/blob/main/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install "numpy<2" scikit-surprise --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached scipy-1.17.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached scipy-1.17.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (35.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.3
    Uninstalling joblib-1.5.3:
      Successfully uninstalled joblib-1.5.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.17.0
    Uninstalling scipy-1.17.0:
      Suc

In [2]:
from surprise import SVD, Dataset, Reader

In [3]:

from scipy.sparse import csr_matrix

In [4]:
import pandas as pd
import numpy as np
import pickle
from difflib import get_close_matches,SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast

In [5]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Load all datasets from Kaggle
credits = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rounakbanik/the-movies-dataset",
    "credits.csv"
)

keywords = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rounakbanik/the-movies-dataset",
    "keywords.csv"
)

movies_metadata = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rounakbanik/the-movies-dataset",
    "movies_metadata.csv",
    pandas_kwargs={'low_memory': False}  # Fix the mixed types warning
)

ratings = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rounakbanik/the-movies-dataset",
    "ratings.csv"
)

links = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "rounakbanik/the-movies-dataset",
    "links.csv"
)

/tmp/ipython-input-2261171664.py:6: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  credits = kagglehub.load_dataset(


Using Colab cache for faster access to the 'the-movies-dataset' dataset.


/tmp/ipython-input-2261171664.py:12: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  keywords = kagglehub.load_dataset(


/tmp/ipython-input-2261171664.py:18: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  movies_metadata = kagglehub.load_dataset(


Using Colab cache for faster access to the 'the-movies-dataset' dataset.


/tmp/ipython-input-2261171664.py:25: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  ratings = kagglehub.load_dataset(


Using Colab cache for faster access to the 'the-movies-dataset' dataset.


/tmp/ipython-input-2261171664.py:31: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  links = kagglehub.load_dataset(


In [6]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [7]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [10]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [11]:
links.isna().sum()

,0
movieId,0
imdbId,0
tmdbId,219


## Merging Dataframes

In [12]:
ratings.isna().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [13]:
links.dropna(inplace=True)
links.isna().sum()

,0
movieId,0
imdbId,0
tmdbId,0


In [14]:
print(links.dtypes)
print()
print(ratings.dtypes)


movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


In [15]:
links['tmdbId'] = links['tmdbId'].astype(int)
ratings = ratings.merge(links[['movieId', 'tmdbId']], on='movieId')
ratings = ratings.rename(columns={'tmdbId': 'id'})
ratings.head()

,userId,movieId,rating,timestamp,id
0,1,110,1.0,1425941529,197
1,1,147,4.5,1425942435,10474
2,1,858,5.0,1425941523,238
3,1,1221,5.0,1425941546,240
4,1,1246,5.0,1425941556,207


In [16]:
# Convert id columns to numeric
credits['id'] = pd.to_numeric(credits['id'], errors="coerce")
keywords['id'] = pd.to_numeric(keywords['id'], errors="coerce")
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors="coerce")

# Drop rows with NaN ids
credits = credits.dropna(subset=['id'])
keywords = keywords.dropna(subset=['id'])
movies_metadata = movies_metadata.dropna(subset=['id'])

# Merge all dataframes
movies_df = credits.merge(keywords, how="left", on="id").merge(movies_metadata, how="left", on="id")

movies_df.head()

,cast,crew,id,keywords,adult,belongs_to_collection,budget,genres,homepage,imdb_id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,tt0113228,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0113041,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Data Preprocessing

In [17]:
movies_df.isna().sum()

,0
cast,0
crew,0
id,0
keywords,0
adult,0
belongs_to_collection,42054
budget,0
genres,0
homepage,38619
imdb_id,17


In [18]:
movies_df = movies_df[movies_df['vote_count'] > 30]
selected_columns = ['cast', 'genres','keywords','title','overview']
# its recommended in NLP and recommenders to fill null vaules with ""
for col in selected_columns:
    movies_df[col] = movies_df[col].fillna("")

movies_df.isna().sum()

,0
cast,0
crew,0
id,0
keywords,0
adult,0
belongs_to_collection,9945
budget,0
genres,0
homepage,8982
imdb_id,2


In [19]:
print(movies_df.duplicated().sum())
print(movies_df['title'].duplicated().sum())

198
645


In [20]:
movies_df["title"].value_counts()

,count
title,
The Congress,8
Le Samouraï,8
Force Majeure,8
Pokémon: Spell of the Unknown,8
Pokémon 4Ever: Celebi - Voice of the Forest,8
...,...
Ride the High Country,1
The Ballad of Cable Hogue,1
The Haunted Palace,1


In [21]:
movies_df.drop_duplicates(inplace=True)
movies_df.drop_duplicates(subset="title", inplace=True)
movies_df.reset_index(drop=True, inplace=True)

print(movies_df.duplicated().sum())
print(movies_df['title'].duplicated().sum())

0
0


In [22]:
def extract_names(column_data):
    try:
        data = ast.literal_eval(column_data)
        return ", ".join([i['name'] for i in data])  # ← Changed to ", "
    except:
        return ""

# Apply to movies_df
movies_df['cast'] = movies_df['cast'].apply(extract_names)
movies_df['genres'] = movies_df['genres'].apply(extract_names)
movies_df['keywords'] = movies_df['keywords'].apply(extract_names)


In [23]:
print(movies_df[['title', 'cast', 'genres']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                                cast  \
0  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...   
1  Robin Williams, Jonathan Hyde, Kirsten Dunst, ...   
2  Walter Matthau, Jack Lemmon, Ann-Margret, Soph...   
3  Whitney Houston, Angela Bassett, Loretta Devin...   
4  Steve Martin, Diane Keaton, Martin Short, Kimb...   

                       genres  
0   Animation, Comedy, Family  
1  Adventure, Fantasy, Family  
2             Romance, Comedy  
3      Comedy, Drama, Romance  
4                      Comedy  


In [24]:
# Use .copy() to avoid SettingWithCopyWarning
movies_df = movies_df.copy()

# Now create combined_text
movies_df['combined_text'] = (movies_df['genres'] + " ") * 5 + \
                              (movies_df['cast'] + " ") * 3 + \
                              (movies_df['keywords'] + " ") * 5 + \
                              (movies_df['title'] + " ") * 1 + \
                              (movies_df['overview'].fillna("") + " ") * 2

In [25]:
movies_df['combined_text'].iloc[0]

"Animation, Comedy, Family Animation, Comedy, Family Animation, Comedy, Family Animation, Comedy, Family Animation, Comedy, Family Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life jealousy, toy, boy, friendship, friends, rivalry, boy next door, new toy, toy comes to life jealousy, toy, boy, friendship, friends, riv

In [25]:
import re
from nltk.stem import PorterStemmer

def text_preprocessing_advanced(text):

    # Lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r"[^a-z\s]", '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Optional: Stemming (reduces words to root form)
    # "running" → "run", "movies" → "movi"
    stemmer = PorterStemmer()
    words = text.split()
    text = ' '.join([stemmer.stem(word) for word in words])

    return text

movies_df["combined_text"] = movies_df["combined_text"].apply(text_preprocessing_advanced)

In [26]:
with open('cleaned_df.pkl','wb') as f:
    pickle.dump(movies_df,f)

## Content Based Filtering

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words='english',  # Removes common words
    min_df=2,              # Word must appear in at least 2 movies
    ngram_range=(1, 2)     # Includes single words and pairs
)

tfidf_matrix = tfidf.fit_transform(movies_df['combined_text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [28]:
print(f"Cosine similarity matrix shape: {cosine_sim.shape}")
print(f"Similarity between movie 0 and movie 1: {cosine_sim[0][1]}")

Cosine similarity matrix shape: (11734, 11734)
Similarity between movie 0 and movie 1: 0.01571976688766283


In [29]:
def content_based_recommendation(
    title,
    cosine_sim=cosine_sim,
    df=movies_df,
    top_n=10,
    with_scores=False
):
    try:
        idx = df[df['title'] == title].index[0]

        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # exclude itself
        sim_scores = sim_scores[1:top_n + 1]

        movie_titles = df.iloc[[i[0] for i in sim_scores]]['title'].values
        scores = [i[1] for i in sim_scores]

        if with_scores:
            return list(zip(movie_titles, scores))
        else:
            return list(movie_titles)

    except IndexError:
        return []


In [30]:
recommendations = content_based_recommendation('Toy Story', top_n=10, with_scores=True)
recommendations

[('Toy Story 2', 0.42355592240068435),
 ('Toy Story 3', 0.3551249360161908),
 ('Toy Story of Terror!', 0.3260295477194195),
 ('Small Soldiers', 0.2797326748695501),
 ('Toy Story That Time Forgot', 0.2752883748545194),
 ("Child's Play 2", 0.24325709396763803),
 ('Partysaurus Rex', 0.23881371399553186),
 ('Small Fry', 0.21843666671280182),
 ('Toys', 0.2027806369298486),
 ('Hawaiian Vacation', 0.2003157996575639)]

## Collaborative Based Filtering

In [31]:
ratings

,userId,movieId,rating,timestamp,id
0,1,110,1.0,1425941529,197
1,1,147,4.5,1425942435,10474
2,1,858,5.0,1425941523,238
3,1,1221,5.0,1425941546,240
4,1,1246,5.0,1425941556,207
...,...,...,...,...,...
26010781,270896,58559,5.0,1257031564,155
26010782,270896,60069,5.0,1257032032,10681
26010783,270896,63082,4.5,1257031764,12405
26010784,270896,64957,4.5,1257033990,4922


In [34]:
def train_surprise_model(ratings_df, n_components=50):
    # Reader expects columns in: user, item, rating
    reader = Reader(rating_scale=(ratings_df['rating'].min(), ratings_df['rating'].max()))
    data = Dataset.load_from_df(ratings_df[['userId', 'id', 'rating']], reader)

    trainset = data.build_full_trainset()

    model = SVD(n_factors=n_components, random_state=42)
    model.fit(trainset)

    return model, trainset, None, None


In [ ]:
svd_model, trainset, _, _ = train_surprise_model(ratings, n_components=50)

import pickle

# Save model
with open('svd_collab_model.pkl', 'wb') as f:
    pickle.dump((svd_model, trainset, None, None), f)

print("✓ Model saved successfully!")

In [ ]:
def collaborative_recommendation_svd(user_id, num_recommendations=100):
    # Use the globals as per your structure
    global svd_model, trainset, movies_df

    try:
        u_inner_id = trainset.to_inner_uid(user_id)
    except ValueError:
        print(f"User {user_id} not found!")
        return []

    rated_inner_ids = [i for (i, _) in trainset.ur[u_inner_id]]
    rated_item_ids = [trainset.to_raw_iid(i) for i in rated_inner_ids]

    all_movie_ids = movies_df['id'].unique()
    recommendations = []

    for m_id in all_movie_ids:
        if m_id not in rated_item_ids:
            pred = svd_model.predict(user_id, m_id).est
            recommendations.append((m_id, pred))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_n = recommendations[:num_recommendations]


    title_map = dict(zip(movies_df['id'], movies_df['title']))
    final_recs = [(title_map.get(m_id, "Unknown"), score) for m_id, score in top_n]

    return final_recs

In [ ]:
test_user_id = ratings["userId"].iloc[0]
recs = collaborative_recommendation_svd(test_user_id, num_recommendations=5)
print("Recommendations:", recs)
